In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics

import os, sys
from time import time

from phm08ds.models import experiment

## Load Dataset

In [2]:
folderpath = '../../../data/interim/'
data_op_1 = pd.read_csv(folderpath + 'data_op_01.csv')
data_op_1.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Sensor_15,Operational_condition,Health_state
2,1,3,34.9986,0.8401,60.0,1368.17,1122.49,5.48,2222.86,183.26,2387.95,0.02,1,1
11,1,12,35.0029,0.8413,60.0,1360.54,1130.69,5.48,2222.88,183.41,2387.95,0.02,1,1
23,1,24,34.9987,0.8400,60.0,1359.28,1131.95,5.48,2222.88,183.36,2387.94,0.02,1,2
38,1,39,35.0010,0.8400,60.0,1370.11,1126.43,5.48,2222.88,183.45,2387.95,0.02,1,2
40,1,41,35.0018,0.8400,60.0,1359.77,1129.48,5.48,2222.89,182.97,2387.98,0.02,1,2


## Data preprocessing

### Get rid of informations there are not sensor readings

Wang (2008) reports Sensor 15 has importat information. However, there are no relevant informations of this sensor. The data seems to be corrupted like this:

Let's remove it from our database creating an object transformer.

In [3]:
from phm08ds.features.feature_selection import RemoveSensor

tf_remove_sensor_15 = RemoveSensor(sensors=[15])
data_op_1 = tf_remove_sensor_15.fit_transform(data_op_1)
data_op_1.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Operational_condition,Health_state
2,1,3,34.9986,0.8401,60.0,1368.17,1122.49,5.48,2222.86,183.26,2387.95,1,1
11,1,12,35.0029,0.8413,60.0,1360.54,1130.69,5.48,2222.88,183.41,2387.95,1,1
23,1,24,34.9987,0.8400,60.0,1359.28,1131.95,5.48,2222.88,183.36,2387.94,1,2
38,1,39,35.0010,0.8400,60.0,1370.11,1126.43,5.48,2222.88,183.45,2387.95,1,2
40,1,41,35.0018,0.8400,60.0,1359.77,1129.48,5.48,2222.89,182.97,2387.98,1,2


Before feeding to the classifier, let's remove unwanted information, such as unit, time_step and operational settings.

In [4]:
from phm08ds.features.feature_selection import RemoveInfo

tf_remove_info = RemoveInfo()

data_with_features = tf_remove_info.fit_transform(data_op_1)
data_with_features.head()

,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Health_state
2,1368.17,1122.49,5.48,2222.86,183.26,2387.95,1
11,1360.54,1130.69,5.48,2222.88,183.41,2387.95,1
23,1359.28,1131.95,5.48,2222.88,183.36,2387.94,2
38,1370.11,1126.43,5.48,2222.88,183.45,2387.95,2
40,1359.77,1129.48,5.48,2222.89,182.97,2387.98,2


We need to normalize our data. Let's use Z-score standardization.

In [5]:
from sklearn.preprocessing import StandardScaler

tf_std_scaller = preprocessing.StandardScaler()
data_with_features_std = tf_std_scaller.fit_transform(data_with_features.drop(labels='Health_state', axis=1))
data_with_features_std

array([[ 2.18753538e-01, -1.28328598e+00, -1.00000000e+00,
        -5.61707703e-01,  6.68924330e-01, -5.45027910e-01],
       [-1.13058311e+00, -1.71913583e-01, -1.00000000e+00,
        -4.90969556e-01,  1.06520396e+00, -5.45027910e-01],
       [-1.35340934e+00, -1.14172757e-03, -1.00000000e+00,
        -4.90969556e-01,  9.33110750e-01, -5.78004106e-01],
       ...,
       [ 8.83695318e-01,  9.06930839e-01, -1.00000000e+00,
        -1.12761287e+00, -1.10112469e+00, -1.40240900e+00],
       [ 1.05523615e+00,  9.55722798e-01, -1.00000000e+00,
        -1.26908916e+00, -1.60307888e+00, -1.13859943e+00],
       [ 2.31968661e+00,  1.83397806e+00, -1.00000000e+00,
        -1.69351804e+00, -2.58056864e+00, -1.83109955e+00]])

In [6]:
labels = np.array(data_with_features['Health_state'])
labels

array([1, 1, 2, ..., 4, 4, 4])

# Classification steps

## Load Experiment model

In [7]:
from phm08ds.models import experiment

## Define classifiers and its specifications

In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [9]:
# SVM
svm_linear_clf = SVC(kernel='linear')
svm_rbf_clf = SVC(kernel='rbf')
svm_poly_clf = SVC(kernel='poly')
svm_sigmoid_clf = SVC(kernel='sigmoid')

## Put all clf in a dictionary:

In [10]:
classifiers = {'SVM-Linear': svm_linear_clf, 'SVM-RBF': svm_rbf_clf, 'SVM-Poly': svm_poly_clf, 'SVM-Sigmoid': svm_sigmoid_clf}

Since we are using SVM and MLP we need to extract all power from those methods. Let's perform a Random Search to parameters optimizations.

### Hyperparameter tunning

In [11]:
from sklearn.model_selection import RandomizedSearchCV

random_search = dict((k,[]) for k in classifiers.keys())

In [12]:
param_dist_dict = {
                   'SVM-Linear': {'C': [2**i for i in range(-5,15)]},
                   'SVM-RBF': {'gamma': [2**i for i in range(-15,3)], 'C': [2**i for i in range(-5,15)]},
                   'SVM-Poly': {'gamma': [2**i for i in range(-15,3)], 'C': [2**i for i in range(-5,15)]},
                   'SVM-Sigmoid': {'gamma': [2**i for i in range(-15,3)], 'C': [2**i for i in range(-5,15)]}
                  }

In [ ]:
for clf in param_dist_dict.keys():
    start = time()
    random_search[clf] = RandomizedSearchCV(classifiers[clf], param_dist_dict[clf], cv=10, n_iter=20, verbose=5, n_jobs=100, scoring='accuracy')
    random_search[clf].fit(data_with_features_std, labels)
    print('Elapsed time:')
    print(time() - start)

Fitting 10 folds for each of 20 candidates, totalling 200 fits
[CV] C=0.03125 .......................................................
[CV] C=0.03125 .......................................................
[CV] C=0.03125 .......................................................
[CV] C=0.03125 .......................................................
[CV] C=0.03125 .......................................................
[CV] C=0.03125 .......................................................
[CV] C=0.03125 .......................................................
[CV] C=0.03125 .......................................................
[CV] C=0.03125 .......................................................
[CV] C=0.03125 .......................................................
[CV] C=0.0625 ........................................................
[CV] C=0.0625 ........................................................
[CV] C=0.0625 ........................................................
[CV] C=0.0625 

[Parallel(n_jobs=100)]: Done  42 out of 200 | elapsed:  1.4min remaining:  5.1min


[CV] C=512 ...........................................................
[CV] ................. C=0.25, score=0.6311953352769679, total=  48.8s
[CV] C=512 ...........................................................
[CV] .................. C=0.5, score=0.6005830903790087, total=  48.7s
[CV] .................. C=0.25, score=0.641399416909621, total=  48.8s
[CV] .................. C=0.5, score=0.6154970760233918, total=  48.8s
[CV] C=512 ...........................................................
[CV] .................. C=0.5, score=0.6034985422740525, total=  48.9s
[CV] C=512 ...........................................................
[CV] C=512 ...........................................................
[CV] C=512 ...........................................................
[CV] .................. C=0.5, score=0.6049562682215743, total=  50.1s
[CV] C=512 ...........................................................
[CV] .................. C=0.5, score=0.6428571428571429, total=  49.6s
[CV] .

[Parallel(n_jobs=100)]: Done  83 out of 200 | elapsed:  2.7min remaining:  3.8min


[CV] .................... C=8, score=0.6428571428571429, total= 2.0min
[CV] .................... C=8, score=0.6142649199417758, total= 2.0min
[CV] C=8192 ..........................................................
[CV] C=8192 ..........................................................
[CV] .................... C=8, score=0.6064139941690962, total= 2.0min
[CV] C=8192 ..........................................................
[CV] .................... C=8, score=0.6034985422740525, total= 2.1min
[CV] C=8192 ..........................................................
[CV] .................... C=8, score=0.6154970760233918, total= 2.0min
[CV] C=8192 ..........................................................
[CV] ..................... C=8, score=0.618978102189781, total= 2.1min
[CV] C=8192 ..........................................................
[CV] .................... C=8, score=0.6200873362445415, total= 2.1min
[CV] C=8192 ..........................................................
[CV] .

## Savel results, models and pipeline to a .pkl file 

In [ ]:
from sklearn.pipeline import Pipeline

data_preprocessing = Pipeline([('remove_sensor_15', tf_remove_sensor_15),
                               ('remove_info', tf_remove_info),
                               ('std_scaler', tf_std_scaller)
                              ])

In [ ]:
experiment.save_models(random_search, name='clf_svm')
experiment.save_pipeline(data_preprocessing)